A notebook to compute 400-member prior=truth recons with different draws for all three models

In [1]:
# put the directory path to your LMR repository here
import sys
sys.path.append("/Users/dan/Desktop/LMR_py3/")
#!cd /Users/dan/Desktop/LMR_py3

# prefix for figure filename
#fig_prefix='prior_truth_'

In [2]:
import os
os.chdir('/Users/dan/Desktop/LMR_py3')
import LMR_lite_utils as LMRlite
import LMR_utils
import LMR_config
import numpy as np
import os,sys
from time import time
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
%matplotlib inline
import cartopy.util as cutil
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd


Loading information from datasets.yml
Loading information from grid_def.yml


In [3]:
# Load and interpolate all of the priors I want to use as truth

def mk_ad(lmr_names,model_names):
    
    # Initialize an array of various interpolated model fields
    ad = []

    for ii in np.arange(len(lmr_names)):
        dd = {}
        cfile = './configs/config.yml.nullspace.'+lmr_names[ii]
        yaml_file = os.path.join(LMR_config.SRC_DIR,cfile)
        cfg = LMRlite.load_config(yaml_file)

        X, Xb_one = LMRlite.load_prior(cfg)
        Xbp = Xb_one - Xb_one.mean(axis=1,keepdims=True)

        # check if config is set to regrid the prior
        if cfg.prior.regrid_method:
            print('regridding prior...')
            # this function over-writes X, even if return is given a different name
            [X,Xb_one_new] = LMRlite.prior_regrid(cfg,X,Xb_one,verbose=False)
        else:
            X.trunc_state_info = X.full_state_info

        Xb_one = Xb_one_new
        Xbp = Xb_one - Xb_one.mean(axis=1,keepdims=True)

        dd['X']            = X
        dd['Xbp']          = Xbp
        dd['name']         = model_names[ii]
        dd['lmr_name']     = lmr_names[ii]
        dd['grid']         = LMRlite.Grid(X)
        dd['prox_manager'] = LMRlite.load_proxies(cfg)
        dd['numprox']      = len(LMRlite.load_proxies(cfg).all_proxies) 

        print(ii)
        print(dd['name'])

        ad.append(dd)
        
    return ad



In [4]:
def mk_pproxies(X,Xbp,prox_manager,SNR,grid,seed=0):

    """
    Construct pseudoproxies 
    """
    
    [_,nens] = Xbp.shape

    numprox = len(prox_manager.ind_assim)

    vY = np.zeros([numprox,nens])
    vR = []
    vP = []
    
    np.random.seed(seed)

    for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
        # get grid indices
        tmp = grid.lat[:,0]-Y.lat
        itlat = np.argmin(np.abs(tmp))
        tmp = grid.lon[0,:]-Y.lon
        itlon = np.argmin(np.abs(tmp))
        npos = itlat*grid.nlon + itlon

        # Noise amplitude corresponding to SNR by stdev
        sig = np.std(Xbp[npos,:])
        #print(sig)
        #print(sig/SNR)
        # Make pproxies
        #import pdb
        #pdb.set_trace()
        randts = np.random.randn(nens,)
        randtsn = randts/np.std(randts)
        vY[proxy_idx,:] = Xbp[npos,:] + randtsn*sig/SNR
        vR.append((sig/SNR)**2)
        vP.append(proxy_idx)
        
    np.random.seed(None)

    return vY, vR, vP

In [5]:
def mk_yes(X,Xbp,prox_manager,grid):
    
    [_,nens] = Xbp.shape
    numprox = len(prox_manager.ind_assim)
    vYe = np.zeros([numprox,nens])
    vYe_coords = np.zeros([numprox,2])

    for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
        # get grid indices
        tmp = grid.lat[:,0]-Y.lat
        itlat = np.argmin(np.abs(tmp))
        tmp = grid.lon[0,:]-Y.lon
        itlon = np.argmin(np.abs(tmp))
        npos = itlat*grid.nlon + itlon
        # the ensemble prior estimates
        vYe[proxy_idx,:] = Xbp[npos,:]
        vYe_coords[proxy_idx,:] = X.coords[npos,:]

    return vYe, vYe_coords

In [6]:
'''
Set up and run prior-truth experiments.

ad is the dictionary of model output (all 1000 years for all)
pmi and tmi are the indices of models used for these experiments
Nensp and Nenst are the number of ensemble members used for prior and truth expectations
Nrealp and Nrealt are the number of realizations of prior and truth draws
'''
def mk_cdd(ad,pmi,tmi,Nensp,Nenst,SNR,LOCRAD,seed=None):
    
    np.random.seed(seed)
    # Draw random truth years
    tinds = np.random.choice(1000, Nenst, replace=False)
    print(tinds)

    # Next: varying random draws of prior ens that are different from the truth ens
    rem   = np.setdiff1d(np.arange(1000),tinds)

    remi  = np.random.choice(range(len(rem)), Nensp, replace=False)
    pinds = rem[remi]
    
    np.random.seed(None)

    # Call routine to generate a prior-truth experiment
    c, Xbpt = mk_pt(ad,pmi,tmi,SNR,LOCRAD,pinds,tinds)

    print('Done!') 
    return c, Xbpt

In [24]:
'''
Set up and run prior-truth experiments. These use perfect priors in the sense that 
the year indices are same for prior and truth, so that if the models are the same, 
the prior and truth ensembles will be equal.

ad is the dictionary of model output (all 1000 years for all)
pmi and tmi are the indices of models used for these experiments
Nensp and Nenst are the number of ensemble members used for prior and truth expectations
Nrealp and Nrealt are the number of realizations of prior and truth draws
'''
def mk_cdd_perfect(ad,pmi,tmi,Nensp,Nenst,SNR,LOCRAD,seed=None):
    
    np.random.seed(seed)
    # Draw random truth years
    tinds = np.random.choice(1000, Nenst, replace=False)
    print(tinds)

    # Next: set prior inds equal to truth inds for perfect prior
    pinds = tinds
    
    np.random.seed(None)

    # Call routine to generate a prior-truth experiment
    c, Xbpt = mk_pt(ad,pmi,tmi,SNR,LOCRAD,pinds,tinds)

    print('Done!') 
    return c, Xbpt

In [7]:
'''
Run prior-truth experiments
'''

def mk_pt(ad,pmi,tmi,SNR,LOCRAD,pinds,tinds):
    
    Nenst = len(tinds)
    Nensp = len(pinds)

    Xbpp  = ad[pmi]['Xbp'][:,pinds]-ad[pmi]['Xbp'][:,pinds].mean(axis=1,keepdims=True)
    Xbpt  = ad[tmi]['Xbp'][:,tinds]-ad[tmi]['Xbp'][:,tinds].mean(axis=1,keepdims=True)
    
    # Compute effective observations
    vYe, vYe_coords = mk_yes(ad[pmi]['X'],Xbpp,ad[pmi]['prox_manager'],ad[pmi]['grid'])

    # Make pseudoproxies
    vY, vR, vP = mk_pproxies(ad[tmi]['X'],Xbpt,ad[tmi]['prox_manager'],SNR,ad[tmi]['grid'])

    # Loop over ensemble members in truth. f are the reconstructions.
    fp = np.empty([ad[tmi]['grid'].nlon*ad[tmi]['grid'].nlat,Nenst])

    for kk in np.arange(Nenst):

        if LOCRAD==0.:
            # Use the optimal solver
            f,Xa,_ = LMRlite.Kalman_optimal(vY[:,kk],vR,vYe,Xbpp,verbose=False)
        else:

            # Load the config file corresponding to the prior
            cfile = './configs/config.yml.nullspace.'+ad[pmi]['lmr_name']
            yaml_file = os.path.join(LMR_config.SRC_DIR,cfile)
            cfg = LMRlite.load_config(yaml_file)

            # change the localization radius in the config file
            cfg_params = LMR_utils.param_cfg_update('core.loc_rad',LOCRAD)
            cfg_new = LMR_config.Config(**cfg_params)
            
            # Use the square root solver
            f,Xa = LMRlite.Kalman_ESRF(cfg_new,vY[:,kk],vR,vYe,Xbpp,X=ad[pmi]['X'],vYe_coords=vYe_coords,verbose=False)

        fp[:,kk]    = f

    # A few calculations...

    lat             = ad[tmi]['grid'].lat
    rmse            = np.mean((Xbpt-fp)**2,1)**.5
    gm_rmse         = np.sum(np.cos(np.deg2rad(lat.ravel()))*rmse**2)/np.sum(np.cos(np.deg2rad(lat)))

    # Store information in a dictionary for this prior-truth pair
    c = {}
    c['locrad']     = LOCRAD
    c['snr']        = SNR
    c['prior_name'] = ad[pmi]['name']
    c['truth_name'] = ad[tmi]['name']
    c['pind']       = pmi
    c['tind']       = tmi
    c['recon']      = fp
    c['rmse']       = rmse
    c['grid']       = ad[tmi]['grid']
    c['gm_rmse']    = gm_rmse
    c['tinds']      = tinds
    c['pinds']      = pinds
    c['Nensp']      = Nensp
    c['Nenst']      = Nenst

    return c, Xbpt

# Begin experiments!

In [8]:
# NB that these are all generated using the same seed (by virtue of multi_seed)
lmr_names = ['ccsm4_last_millenium.1000',
             'mpi-esm-p_last_millenium.1000',
             'hadcm3_last_millenium.1000']

model_names = ['CCSM4','MPI-ESM','HadCM3']

ad=mk_ad(lmr_names,model_names)

Checking configuration ... 
OK!
Reading file:  /Users/dan/Desktop/LMR_py3/data/model/ccsm4_last_millenium/tas_sfc_Amon_CCSM4_past1000_085001-185012.nc
(12012, 192, 288)
indlat= 0  indlon= 1
Anomalies provided as the prior: Removing the temporal mean (for every gridpoint)...
tas : Global(monthly): mean= 8.072375e-07  , std-dev= 1.8899411
Averaging over month sequence: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
tas : Global(time-averaged): mean= 4.4424884352419226e-08  , std-dev= 0.8317386411161235
 
State vector information:
Nx = 55296
state_vect_info= {'tas_sfc_Amon': {'pos': (0, 55295), 'spacecoords': ('lat', 'lon'), 'spacedims': (192, 288), 'vartype': '2D:horizontal'}}
Random selection of 1000 ensemble members
regridding prior...
0 55295
(55296, 1000)
(55296, 2)
(55296, 2)
tas_sfc_Amon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 192 288
=> Full array:      -11.247562408447266 8.779441833496094 0.0002476248215526498 0.8317324770123996
=> Truncated array: -11.1011901257028

In [11]:
def process_input(ad,pmiv,tmiv,Nensp,Nenst,SNR,savename,LOCRAD):

    cdd  = []
    Xbpt_dd = []
    
    for ii in np.arange(len(pmiv)):
        pmi = pmiv[ii]
        tmi = tmiv[ii]
        c, Xbpt   = mk_cdd(ad,pmi,tmi,Nensp,Nenst,SNR,LOCRAD,seed=0)
        
        cdd.append(c)
        Xbpt_dd.append(Xbpt)
    
    fullsn = savename + '_locrad_' + str(LOCRAD)+'_SNR_'+str(SNR)

    np.save(fullsn,cdd)
    np.save(fullsn+'_ad',Xbpt_dd)
   

In [25]:
def process_input_perfect(ad,pmiv,tmiv,Nensp,Nenst,SNR,savename,LOCRAD):

    cdd  = []
    Xbpt_dd = []
    
    for ii in np.arange(len(pmiv)):
        pmi = pmiv[ii]
        tmi = tmiv[ii]
        c, Xbpt   = mk_cdd_perfect(ad,pmi,tmi,Nensp,Nenst,SNR,LOCRAD,seed=0)
        
        cdd.append(c)
        Xbpt_dd.append(Xbpt)
    
    fullsn = savename + '_locrad_' + str(LOCRAD)+'_SNR_'+str(SNR)

    np.save(fullsn,cdd)
    np.save(fullsn+'_ad',Xbpt_dd)
   

In [32]:
# Run in parallel

#SNR     = 1.6
Nensp   = 400
Nenst   = 400
#pmiv    = [0,1,2]
#tmiv    = [0,1,2]
pmiv    = [0,1,2,0,1,2,0,1,2]
tmiv    = [0,0,0,1,1,1,2,2,2]
#pmiv     = [0]
#tmiv     = [1]
savename = '/Users/dan/Desktop/Nullspace/pt_out/PAGES2k_400_rand_draws_one_truth'

LOCRAD  = 25000.
#SNRs    = [0.005,0.01,0.05,0.1,0.2,0.4,.8,1.6,3.2,6.4,12.8,25.,50.]
SNRs    = [0.05,0.4,1.6,3.2,6.4,25.]
#SNRs = [0.4]
#SNRs    = [0.005,0.001,25.,50,100.]
#SNRs    = [1000.,2000.]

from joblib import Parallel, delayed
import multiprocessing
#num_cores = multiprocessing.cpu_count()
num_cores = 8
nj = len(SNRs)
Parallel(n_jobs=num_cores)(delayed(process_input)(ad,pmiv,tmiv,Nensp,Nenst,SNRs[i],savename,LOCRAD) for i in np.arange(nj))

[None, None, None, None, None, None]

In [31]:
# Run in parallel: perfect cases

#SNR     = 1.6
Nensp   = 400
Nenst   = 400
#pmiv    = [0,1,2]
#tmiv    = [0,1,2]
pmiv    = [0,1,2]
tmiv    = [0,1,2]
savename = '/Users/dan/Desktop/Nullspace/pt_out/PAGES2k_400_rand_draws_one_truth_perfect'

LOCRAD  = 0.0
#SNRs    = [0.2,0.1,0.05,0.01,.8,1.6,3.2,6.4,12.8]
#SNRs = [0.4]
#SNRs    = [0.005,0.001,25.,50.,100.,0.2,0.1,0.05,0.01,.8,1.6,3.2,6.4,12.8,0.4]
SNRs    = [1000.,2000.]

from joblib import Parallel, delayed
import multiprocessing
#num_cores = multiprocessing.cpu_count()
num_cores = 8
nj = len(SNRs)
Parallel(n_jobs=num_cores)(delayed(process_input_perfect)(ad,pmiv,tmiv,Nensp,Nenst,SNRs[i],savename,LOCRAD) for i in np.arange(nj))

[None, None]